In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['PAF']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('platelet activating factor', 3280),
 ('paroxysmal atrial fibrillation', 50),
 ('population attributable fraction', 25),
 ('pure autonomic failure', 15),
 ('acetyl sn glycero 3 phosphocholine', 14),
 ('paroxysmal af', 11),
 ('paf', 10),
 ('posterior auditory field', 8),
 ('polymerase associated factor', 8),
 ('antifungal protein', 7),
 ('platelet activating fac tor', 7),
 ('peak alpha frequency', 6),
 ('platelet activating factor 16', 5),
 ('plasma autofluorescence', 4),
 ('platelet aggregation factor', 4),
 ('plateletactivating factor', 4),
 ('platelet acti vating factor', 4),
 ('polymerase ii associated factor', 4),
 ('positive affect function', 3),
 ('postoperative atrial fibrillation', 3),
 ('platelet activating factor acether', 3),
 ('platelet activat ing factor', 3)]

In [7]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-03 01:31:23] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [8]:
result = [grounding_map, names, pos_labels]

In [9]:
result

[{'acetyl sn glycero 3 phosphocholine': 'CHEBI:CHEBI:52450',
  'antifungal protein': 'ungrounded',
  'paf': 'ungrounded',
  'paroxysmal af': 'MESH:D001281',
  'paroxysmal atrial fibrillation': 'MESH:D001281',
  'peak alpha frequency': 'ungrounded',
  'plasma autofluorescence': 'ungrounded',
  'platelet acti vating factor': 'CHEBI:CHEBI:52450',
  'platelet activat ing factor': 'CHEBI:CHEBI:52450',
  'platelet activating fac tor': 'CHEBI:CHEBI:52450',
  'platelet activating factor': 'CHEBI:CHEBI:52450',
  'platelet activating factor 16': 'CHEBI:CHEBI:52450',
  'platelet activating factor acether': 'CHEBI:CHEBI:52450',
  'platelet aggregation factor': 'CHEBI:CHEBI:52450',
  'plateletactivating factor': 'CHEBI:CHEBI:52450',
  'polymerase associated factor': 'FPLX:Paf1/RNA_polymerase_II_complex',
  'polymerase ii associated factor': 'FPLX:Paf1/RNA_polymerase_II_complex',
  'population attributable fraction': 'ungrounded',
  'positive affect function': 'ungrounded',
  'posterior auditory fie

In [6]:
grounding_map, names, pos_labels = [{'acetyl sn glycero 3 phosphocholine': 'CHEBI:CHEBI:52450',
  'antifungal protein': 'ungrounded',
  'paf': 'ungrounded',
  'paroxysmal af': 'MESH:D001281',
  'paroxysmal atrial fibrillation': 'MESH:D001281',
  'peak alpha frequency': 'ungrounded',
  'plasma autofluorescence': 'ungrounded',
  'platelet acti vating factor': 'CHEBI:CHEBI:52450',
  'platelet activat ing factor': 'CHEBI:CHEBI:52450',
  'platelet activating fac tor': 'CHEBI:CHEBI:52450',
  'platelet activating factor': 'CHEBI:CHEBI:52450',
  'platelet activating factor 16': 'CHEBI:CHEBI:52450',
  'platelet activating factor acether': 'CHEBI:CHEBI:52450',
  'platelet aggregation factor': 'CHEBI:CHEBI:52450',
  'plateletactivating factor': 'CHEBI:CHEBI:52450',
  'polymerase associated factor': 'FPLX:PAF1_complex',
  'polymerase ii associated factor': 'FPLX:PAF1_complex',
  'population attributable fraction': 'ungrounded',
  'positive affect function': 'ungrounded',
  'posterior auditory field': 'ungrounded',
  'postoperative atrial fibrillation': 'MESH:D001281',
  'pure autonomic failure': 'MESH:D054970'},
 {'CHEBI:CHEBI:52450': '2-O-acetyl-1-O-octadecyl-sn-glycero-3-phosphocholine',
  'MESH:D001281': 'Atrial Fibrillation',
  'FPLX:PAF1_complex': 'PAF1_complex',
  'MESH:D054970': 'Pure Autonomic Failure'},
 ['CHEBI:CHEBI:52450',
  'FPLX:PAF1_complex',
  'MESH:D001281',
  'MESH:D054970']]

In [7]:
excluded_longforms = ['paf']

In [8]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [9]:
additional_entities = {'HGNC:28961': ['PCLAF', ['PAF', 'PAF15']]} 

In [10]:
unambiguous_agent_texts = {}

In [11]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:28961', 'HGNC:28961', 67)]

In [15]:
intersection2

[('CHEBI:CHEBI:52450', 'HGNC:28961', 2),
 ('MESH:D001281', 'HGNC:28961', 0),
 ('MESH:D054970', 'HGNC:28961', 0),
 ('ungrounded', 'HGNC:28961', 0),
 ('FPLX:PAF1_complex', 'HGNC:28961', 0)]

In [16]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [17]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [18]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-30 01:26:06] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-30 01:26:38] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9913494596957035 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [19]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:52450': 2697,
  'MESH:D001281': 54,
  'MESH:D054970': 9,
  'ungrounded': 30,
  'FPLX:PAF1_complex': 9,
  'HGNC:28961': 16},
 'f1': {'mean': 0.991349, 'std': 0.004307},
 'precision': {'mean': 0.988991, 'std': 0.004938},
 'recall': {'mean': 0.994614, 'std': 0.003406},
 'CHEBI:CHEBI:52450': {'f1': {'mean': 0.994839, 'std': 0.002225},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.989742, 'std': 0.004415}},
 'FPLX:PAF1_complex': {'f1': {'mean': 0.466667, 'std': 0.4},
  'pr': {'mean': 0.4, 'std': 0.374166},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'HGNC:28961': {'f1': {'mean': 0.92, 'std': 0.09798},
  'pr': {'mean': 0.866667, 'std': 0.163299},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D054970': {'f1': {'mean': 0.866667, 'std': 0.163299},
  'pr': {'mean': 0.8, 'std': 0.244949},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.687446, 'std': 0.228256},
  'pr': {'mean': 0.566667, 'std': 0.249444},
  'rc': {'mean': 1.0, 'std': 0.

In [20]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [21]:
disamb.dump(model_name, results_path)

In [22]:
print(disamb.info())

Disambiguation model for PAF

Produces the disambiguations:
	2-O-acetyl-1-O-octadecyl-sn-glycero-3-phosphocholine*	CHEBI:CHEBI:52450
	Atrial Fibrillation*	MESH:D001281
	PAF1_complex*	FPLX:PAF1_complex
	PCLAF*	HGNC:28961
	Pure Autonomic Failure*	MESH:D054970

Class level metrics:
--------------------
Grounding                                           	Count	F1     
2-O-acetyl-1-O-octadecyl-sn-glycero-3-phosphocholine*	2697	0.99484
                                 Atrial Fibrillation*	  54	0.93885
                                          Ungrounded	  30	0.68745
                                               PCLAF*	  16	   0.92
                              Pure Autonomic Failure*	   9	0.86667
                                        PAF1_complex*	   9	0.46667

Weighted Metrics:
-----------------
	F1 score:	0.99135
	Precision:	0.98899
	Recall:		0.99461

* Positive labels
See Docstring for explanation



In [23]:
model_to_s3(disamb)

In [32]:
preds = disamb.disambiguate(all_texts.values())

In [35]:
PCLAF_texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0].startswith('HGNC')]

In [37]:
PCLAF_texts[0]

"p15 PAF  (15 kDa, PCNA-associated-factor) was first identified via a yeast two-hybrid screen for proteins that bind to PCNA  [1] . The p15 PAF  protein contains a conserved PCNA-binding motif and when overexpressed competes with p21 WAF  for PCNA binding. Expression analysis revealed upregulation of  p15 PAF  RNA in a range of tumor types  [1] , and an independent microarray study revealed elevated expression in colon tumors  [3] . The  KIAA0101 / p15 PAF  gene maps to human chromosome 15q22.31, a region thought to harbour a tumor suppressor gene for a sporadic form of colorectal cancer  [4,5] . Amplification of 15q22 also correlates with poor survival in colorectal cancer patients  [6] . Recently, the KIAA0101 protein has also been published as OEATC-1, identified as a gene overexpressed in anaplastic thyroid carcinoma-1  [2] .\nSimilarly to p15 PAF  (1), p33ING1b contains a PCNA-binding motif (PIP) and binds to PCNA competitively with p21 WAF   [1,14] . p33ING1b is a specific isofor